In [ ]:
## Análise de diferenças entre os dados - ANOVA, boxplot, t-confidence interval

## CÉLULA 1: Instruções de carregamento do arquivo e bibliotecas a ser importadas

""""
    Para executar uma CÉLULA, pressione o botão "Run" no topo, com a CÉLULA selecionada, ou pressione:
    
    Shift + Enter

-Se seus dados estiverem em um arquivo xlsx utilize a CÉLULA 2

-Se seus dados estiverem em um arquivo csv contendo cabeçalhos, utilize a CÉLULA 3

-Se seus dados estiverem em um arquivo csv sem cabeçalhos, utilize a CÉLULA 4

-Utilize apenas a CÉLULA correta
-Substitua o texto entre aspas na variável "caminho" pelo endereço do arquivo em sua máquina.

EXECUTE ESTA CÉLULA ANTES DE PROSSEGUIR

EXECUTE APENAS UMA CÉLULA POR VEZ

-Caso deseje apagar a saída de uma célula carregada, mas não deseje carregar uma nova saída, vá à aba superior, escolha o ícone
do teclado (open the command palette/ jupyter-notebook command group) e selecione clear cell output.

@author: Marco César Prado Soares, MSc.
Especialista Lean Six Sigma Master Black Belt, Eng. Químico, MSc. Eng. Mecatrônica (instrumentação) 
Marco.Soares@br.ey.com; marcosoares.feq@gmail.com
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## CÉLULA 2: dados em arquivo xlsx

caminho = "D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 2 - ANN regression\ANN.1 - Apartment price analysis\prices_apartments.xlsx"
dataset = pd.read_excel(caminho)

dataset
#SIGA PARA A CÉLULA 5

In [ ]:
## CÉLULA 3: dados em arquivo csv com cabeçalho

caminho = "D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 3 - ANN classification\ANN.Class - 2 - advertising campaign\delivery_app_data.csv"
dataset = pd.read_csv(caminho)

dataset
#SIGA PARA A CÉLULA 5

In [ ]:
## CÉLULA 4: dados em arquivo csv sem cabeçalho

caminho = "D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 2 - ANN regression\ANN.4 - Boston housing price\housing_boston_data.txt"
dataset = pd.read_csv(caminho, delim_whitespace=True, header=None)

#here, the dataset is in a textfile. In these cases, use delim_withespace = True
#or df = pd.read_fwf('output_list.txt')
#since data has no Head, keep header = None
#if there is Head, eliminate ‘header’ from the arguments of the function

dataset
#SIGA PARA A CÉLULA 5

In [ ]:
## CÉLULA 5: Eliminar linhas com entradas nulas

"""
-Utilize este CÉLULA apenas se desejar limpar as entradas nulas da sua base de dados

-Esta CÉLULA eliminará todas as linhas contendo entradas nulas. Caso NÃO DESEJE ISTO, vá direto à CÉLULA 6, sem executar esta
célula.
"""

dataset = dataset.dropna(axis=0)
dataset
#SIGA PARA A CÉLULA 6

# CÉLULA 6: Instruções Gerais

### ATENÇÃO: NÃO EXECUTE ESTA CÉLULA - Ela contém apenas texto explicativo

Este notebook destina-se à análise de diferenças de médias entre conjuntos de dados, construção dos diagramas boxplot e cálculo dos intervalos de confiança das médias (t-confidence intervals). Utilize a função correspondente à sua situação.

## 1) Dados a serem comparados estão todos na mesma coluna

- Neste caso, os dados estão todos na mesma coluna, e uma segunda coluna identifica os diferentes tratamentos. Ou seja: colunas diferentes NÃO correspondem a tratamentos diferentes.
- A função permite avaliar até 6 fatores/tratamentos simultâneos, sem necessidade de expansão do código, e possibilita a construção do boxplot e dos intervalos de confiança das médias.

Siga para as CÉLULAS 7 a 9.
----------------------------------------

_______________________________________________________________________________________________________________________________
_______________________________________________________________________________________________________________________________
_______________________________________________________________________________________________________________________________

## 2) Dados a serem comparados estão separados em colunas distintas

- Neste caso, os dados estão cada um em uma coluna diferente. Ou seja: colunas diferentes CORRESPONDEM a tratamentos diferentes, e cada coluna corresponde a um único tratamento/fator.
- A função permite avaliar até 6 fatores/tratamentos simultâneos, sem necessidade de expansão do código, e possibilita a construção do boxplot e dos intervalos de confiança das médias.

Siga para as CÉLULAS 10 a 12.
----------------------------------------

In [ ]:
## CÉLULA 7: Função para análise de variância (ANOVA) - Verificar se as médias dos resultados correspondentes a diferentes
# tratamentos são ou não diferentes. 
# Neste caso, a função analisa a situação na qual os dados estão todos na mesma coluna, ou seja, uma segunda coluna é utilizada
# para identificar os diferentes tratamentos (por meio de tags/marcadores)
# Colunas diferentes NÃO correspondem a tratamentos diferentes.

#Execute esta CÉLULA para carregar a função

def anova_boxplot_same_column(dataframe, col_y_name, col_factor_name, confidence = 0.95, factor_labels = None, reference_line = None,  plot_boxplot = True, calculate_confidence_interval = True):

        import pandas as pd
        import matplotlib.pyplot as plt
        import numpy as np
        from scipy import stats
        
        name1 = col_y_name
        name2 = col_factor_name
        
        #organizar o dataframe, de modo que os grupos fiquem juntos, e possa ser
        #identificado o numero de fatores (grupos):
        
        # Exemplo de como seria se fosse conhecido o nome da coluna:
        # teste = dataset.sort_values(by=['Lote A'])
        
        dataframe = dataframe.sort_values(by = [name2])
        #os fatores agora estao agrupados
        dataframe = dataframe.reset_index(drop=True)
        #garante que os indices estejam em ordem
        
        y = dataframe[name1]
        #valores y em torno dos quais sera feito o ANOVA
        factors = dataframe[name2]
        n_measurements = len(dataframe)
        #total de dados
        
        grupo = factors[0] #primeira posicao
            
        #vamos encontrar os indices nos quais ocorrem as mudancas de fator
        #e quebrar em varios arrays de acordo com os fatores
            
        indices_of_factors = []
        #lista dos indices de troca de array
        #sao os indices em que ocorre mudanca de fator
            
        i = 1
        while (i < n_measurements):
                
            if (i > (n_measurements-1)):
                        
                #Esta restricao extra impede que se acesse um indice
                #i inexistente
                break
            
            if (grupo != factors[i]):
                    
                indices_of_factors.append(i)
                #grava o indice da mudanca de fator
                #este indice indicara onde quebrar em arrays
                grupo = factors[i]
                    
            i = i + 1    
            
            #total de splits/fatores
            #se a lista indices_of_factors possui apenas um elemento, significa
            #que temos 2 splits (2 fatores diferentes)
            #se existem 2 elementos, temos 3 fatores diferentes
            #logo, o numero de fatores = total de elementos da lista 
            #indices_of_factors + 1
            
        n_factors = len(indices_of_factors) + 1
        #variavel que armazena o total de fatores/splits das listas originais
            
        #CHECAR CONDICOES: O GRAFICO NAO PODE SER GERADO SE n_tags = 1 ou
        #n_tags >=4. Nestas condicoes, a mensagem de erro seria gerada
        #pois o grafico usa variaveis locais criadas para cada condicao
            
        #CHECAR CONDICOES: O GRAFICO NAO PODE SER GERADO SE
        #n_tags >=7. Nestas condicoes, a mensagem de erro seria gerada
        #pois o grafico usa variaveis locais criadas para cada condicao
        
        if (n_factors > 6):
            print("\n") #\n = quebra de linha
            print("Error! Too much factors!\n")
            print("Copy and paste the code to create more factors, or reduce the number of tested factors.\n")
        
        else:
            
            if not (factor_labels is None):
                
                if (len(factor_labels) != n_factors):
                    print("\n") #\n = quebra de linha
                    print("Error! The total of elements on factors_labels list is different from the number of factors.\n")
                    print("Correct it, or keep factors_labels = None\n")
                
                    
            if(n_factors == 1):
                print("\n") #\n = quebra de linha
                print("Warning! Only one factor! The analysis of variance demands at least two factors/groups to be compared. Only the mean confidence interval was calculated.\n")
                
                if (calculate_confidence_interval == True):
                    #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                    n = n_measurements
                    degree_freedom = n-1
                    xbar = y.mean()
                    s = y.std()
                    #valor medio
                    nivel = confidence #nao perder o valor
                    nivel = nivel*100 #registrar em porcentagem
                    confidence = 1 - confidence
                    #alfa da t student bicaudal
                    confidence = confidence/2
                    #alfa/2
                    t = stats.t.ppf((1-confidence), degree_freedom)
                            
                    #intervalo de confianca da media:
                    lim_inf = xbar - t*s/(np.sqrt(n))
                    lim_sup = xbar + t*s/(np.sqrt(n))
                    
                    if not (factor_labels is None):
                        y_lab = factor_labels[0]
                            
                    else:
                        y_lab = factors[0]
                    
                    estatisticas = []
                    y_res = []
                                
                    string1 = "Nível de confiança (em porcentagem) = %.2f" %(nivel)
                    estatisticas.append(string1)
                    y_res.append(None)
                    string2 = "Limite inferior do intervalo de confiança da média"
                    estatisticas.append(string2)
                    y_res.append(lim_inf)
                    texto_extra = "Média amostral: centro do intervalo de confiança"
                    estatisticas.append(texto_extra)
                    y_res.append(xbar)
                    string3 = 'Limite superior do intervalo de confiança da média'
                    estatisticas.append(string3)
                    y_res.append(lim_sup)
                    
                    if not (reference_line is None):
                        string4 = "Valor de referência: %.2f" %(reference_line)
                        estatisticas.append(string4)
                        if (lim_inf < reference_line) and (reference_line < lim_sup):
                            y_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                        else:
                            y_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                    
                    d = {'Valor': estatisticas,
                         y_lab: y_res}
                            
                    confidence_interval = pd.DataFrame(data = d)
                    
                if (plot_boxplot == True):
                            
                    #construir box plot
                    fig, ax = plt.subplots()
                    
                    # rectangular box plot
                    ax.boxplot([y], labels = [y_lab])
                    
                    ax.set_title('Box plot')
                    ax.set(ylabel='Variables')
                    
                    if not (reference_line is None):
                        ax.axhline(reference_line, color = 'red', linestyle = 'dashed', label = 'Reference')
                    
                    #Criar linha horizontal tenue, que apenas guie os olhos se distrair
                    ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey', alpha=0.5)
                    ax.grid(True) #mude para False, caso não deseje ver as linhas de grade      
                    ax.legend()
                    plt.show()
                    
                if (calculate_confidence_interval == True):
                    
                    return confidence_interval
            
            if (2 <= n_factors <= 6): 
                    
                    if not (factor_labels is None):
                
                        if (len(factor_labels) != n_factors):
                            print("\n") #\n = quebra de linha
                            print("Error! The total of elements on factors_labels list is different from the number of factors\n")
                            print("Correct it, or keep factors_labels = None\n")
                    
                        """
                        Vamos primeiro quebrar os valores de acordo com os fatores
                        Para isso, usamos a função numpy.array_split(ary, indices_or_sections) 
                        que quebra a lista ou array 'ary' nas posições indicadas na lista 
                        'indices_or_sections'. A função numpy.split não pode ser usada, pois ela 
                        apenas permite quebra se a lista 'ary' for dividida em sublistas de mesmo
                        tamanho.
                        
                        Ao fim, reconcatemos as listas usando a função 
                        numpy.concatenate((y1, y2, ...), axis=None) que junta as listas em um novo
                        array.
                        
                        Esta função, porém, também tem uma limitação: ela não permite concatenar datas
                        ou horas, que podem constituir o eixo X de uma série temporal.
                        (Esta limitação não existe quando se trata de variáveis float ou strings)
                        
                        Porém, na etapa final, nao precisamos nos preocupar. Como a variavel x
                        não é manipulada, basta tomar o x original (ela porém é separada
                        para indicar os valores corretos na avaliação de se o ponto esta
                        dentro dos limites). O mesmo é feito com as tags, evitando que a
                        presença de hora/data comprometa o código.
                        
                        O X apenas é dividido para permitir a aferição do ponto que passa
                        dos limites de controle ou especificação. Como não há informação
                        obtida pela divisão das tags, elas não são divididas.
                        
                        """
                        
                    if (n_factors == 2):
                        #temos dois subgrupos
                           
                            y1, y2 = np.split(ary = y, indices_or_sections = indices_of_factors, axis=0)
                            y1 = y1.reset_index(drop=True)
                            y2 = y2.reset_index(drop=True)
                            n1 = len(y1)
                            n2 = len(y2)
                            
                            anova = stats.f_oneway(y1, y2)
                            p_val = anova[1]
                            #p-valor do anova
                            
                            if not (factor_labels is None):
                                y1_lab = factor_labels[0]
                                y2_lab = factor_labels[1]
                            
                            else:
                                factors1, factors2 = np.split(ary = factors, indices_or_sections = indices_of_factors, axis=0)
                                factors1 = factors1.reset_index(drop=True)
                                factors2 = factors2.reset_index(drop=True)
                                y1_lab = factors1[0]
                                y2_lab = factors2[0]
                                #procedimento que consome memoria extra: pode
                                #ser evitando fornecendo-se a lista pronta
                            
                            if (calculate_confidence_interval == True):
                                
                                nivel = confidence #nao perder o valor
                                nivel = nivel*100 #registrar em porcentagem
                                confidence = 1 - confidence
                                #alfa da t student bicaudal
                                confidence = confidence/2
                                #alfa/2
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom1 = n1-1
                                xbar1 = np.average(y1) #media
                                s1 = np.std(y1) #desv-pad
                                t1 = stats.t.ppf((1-confidence), degree_freedom1)
                                        
                                #intervalo de confianca da media:
                                lim_inf1 = xbar1 - t1*s1/(np.sqrt(n1))
                                lim_sup1 = xbar1 + t1*s1/(np.sqrt(n1))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom2 = n2-1
                                xbar2 = np.average(y2) #media
                                s2 = np.std(y2) #desv-pad
                                t2 = stats.t.ppf((1-confidence), degree_freedom2)
                                        
                                #intervalo de confianca da media:
                                lim_inf2 = xbar2 - t2*s2/(np.sqrt(n2))
                                lim_sup2 = xbar2 + t2*s2/(np.sqrt(n2))
                                
                                estatisticas = []
                                y1_res = []
                                y2_res = []
                                
                                string1 = "Nível de confiança (em porcentagem) = %.2f" %(nivel)
                                estatisticas.append(string1)
                                y1_res.append(None)
                                y2_res.append(None)
                                string2 = "Limite inferior do intervalo de confiança da média"
                                estatisticas.append(string2)
                                y1_res.append(lim_inf1)
                                y2_res.append(lim_inf2)
                                texto_extra = "Média amostral: centro do intervalo de confiança"
                                estatisticas.append(texto_extra)
                                y1_res.append(xbar1)
                                y2_res.append(xbar2)
                                string3 = 'Limite superior do intervalo de confiança da média'
                                estatisticas.append(string3)
                                y1_res.append(lim_sup1)
                                y2_res.append(lim_sup2)
                                string4 = "p-valor do ANOVA:\nProbabilidade de as médias serem iguais"
                                estatisticas.append(string4)
                                y1_res.append(p_val)
                                y2_res.append(p_val)
                                string5 = "Probabilidade de as médias serem iguais (em %)"
                                estatisticas.append(string5)
                                y1_res.append(p_val*100)
                                y2_res.append(p_val*100)
                                
                                if not (reference_line is None):
                                    string6 = "Valor de referência: %.2f" %(reference_line)
                                    estatisticas.append(string6)
                                    if (lim_inf1 < reference_line) and (reference_line < lim_sup1):
                                        y1_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y1_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf2 < reference_line) and (reference_line < lim_sup2):
                                        y2_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y2_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                
                                d = {'Valor': estatisticas,
                                     y1_lab: y1_res,
                                     y2_lab: y2_res}
                            
                                confidence_interval = pd.DataFrame(data = d)
                            
                            else:
                                d = {'p-valor do ANOVA: probabilidade de as médias serem iguais': [p_val]}
                                     
                                confidence_interval = pd.DataFrame(data = d)
                            
                            if (plot_boxplot == True):
                            
                                #construir box plot
                                fig, ax = plt.subplots()
                                
                                ax.boxplot([y1, y2], labels = [y1_lab, y2_lab])
                                ax.set_title('Box plot')
                                ax.set(ylabel='Variables')
                                
                                if not (reference_line is None):
                                    ax.axhline(reference_line, color = 'red', linestyle = 'dashed', label = 'Reference')
                                
                                #Criar linha horizontal tenue, que apenas guie os olhos se distrair
                                ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey', alpha=0.5)
                                ax.grid(True) #mude para False, caso não deseje ver as linhas de grade      
                                ax.legend()
                                plt.show()
                    
                            return confidence_interval
                        
                    if (n_factors == 3):
                            #temos tres subgrupos
                           
                            y1, y2, y3 = np.split(ary = y, indices_or_sections = indices_of_factors, axis=0)
                            y1 = y1.reset_index(drop=True)
                            y2 = y2.reset_index(drop=True)
                            y3 = y3.reset_index(drop=True)
                            n1 = len(y1)
                            n2 = len(y2)
                            n3 = len(y3)
                            
                            anova = stats.f_oneway(y1, y2, y3)
                            p_val = anova[1]
                            #p-valor do anova
                            
                            if not (factor_labels is None):
                                y1_lab = factor_labels[0]
                                y2_lab = factor_labels[1]
                                y3_lab = factor_labels[2]
                            
                            else:
                                factors1, factors2, factors3 = np.split(ary = factors, indices_or_sections = indices_of_factors, axis=0)
                                factors1 = factors1.reset_index(drop=True)
                                factors2 = factors2.reset_index(drop=True)
                                factors3 = factors3.reset_index(drop=True)
                                y1_lab = factors1[0]
                                y2_lab = factors2[0]
                                y3_lab = factors3[0]
                                #procedimento que consome memoria extra: pode
                                #ser evitando fornecendo-se a lista pronta
                            
                            if (calculate_confidence_interval == True):
                                
                                nivel = confidence #nao perder o valor
                                nivel = nivel*100 #registrar em porcentagem
                                confidence = 1 - confidence
                                #alfa da t student bicaudal
                                confidence = confidence/2
                                #alfa/2
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom1 = n1-1
                                xbar1 = np.average(y1) #media
                                s1 = np.std(y1) #desv-pad
                                t1 = stats.t.ppf((1-confidence), degree_freedom1)
                                        
                                #intervalo de confianca da media:
                                lim_inf1 = xbar1 - t1*s1/(np.sqrt(n1))
                                lim_sup1 = xbar1 + t1*s1/(np.sqrt(n1))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom2 = n2-1
                                xbar2 = np.average(y2) #media
                                s2 = np.std(y2) #desv-pad
                                t2 = stats.t.ppf((1-confidence), degree_freedom2)
                                        
                                #intervalo de confianca da media:
                                lim_inf2 = xbar2 - t2*s2/(np.sqrt(n2))
                                lim_sup2 = xbar2 + t2*s2/(np.sqrt(n2))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom3 = n3-1
                                xbar3 = np.average(y3) #media
                                s3 = np.std(y3) #desv-pad
                                t3 = stats.t.ppf((1-confidence), degree_freedom3)
                                        
                                #intervalo de confianca da media:
                                lim_inf3 = xbar3 - t3*s3/(np.sqrt(n3))
                                lim_sup3 = xbar3 + t3*s3/(np.sqrt(n3))
                                
                                estatisticas = []
                                y1_res = []
                                y2_res = []
                                y3_res = []
                                
                                string1 = "Nível de confiança (em porcentagem) = %.2f" %(nivel)
                                estatisticas.append(string1)
                                y1_res.append(None)
                                y2_res.append(None)
                                y3_res.append(None)
                                string2 = "Limite inferior do intervalo de confiança da média"
                                estatisticas.append(string2)
                                y1_res.append(lim_inf1)
                                y2_res.append(lim_inf2)
                                y3_res.append(lim_inf3)
                                texto_extra = "Média amostral: centro do intervalo de confiança"
                                estatisticas.append(texto_extra)
                                y1_res.append(xbar1)
                                y2_res.append(xbar2)
                                y3_res.append(xbar3)
                                string3 = 'Limite superior do intervalo de confiança da média'
                                estatisticas.append(string3)
                                y1_res.append(lim_sup1)
                                y2_res.append(lim_sup2)
                                y3_res.append(lim_sup3)
                                string4 = "p-valor do ANOVA:\nProbabilidade de as médias serem iguais"
                                estatisticas.append(string4)
                                y1_res.append(p_val)
                                y2_res.append(p_val)
                                y3_res.append(p_val)
                                string5 = "Probabilidade de as médias serem iguais (em %)"
                                estatisticas.append(string5)
                                y1_res.append(p_val*100)
                                y2_res.append(p_val*100)
                                y3_res.append(p_val*100)
                                
                                if not (reference_line is None):
                                    string6 = "Valor de referência: %.2f" %(reference_line)
                                    estatisticas.append(string6)
                                    if (lim_inf1 < reference_line) and (reference_line < lim_sup1):
                                        y1_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y1_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf2 < reference_line) and (reference_line < lim_sup2):
                                        y2_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y2_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf3 < reference_line) and (reference_line < lim_sup3):
                                        y3_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y3_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                
                                d = {'Valor': estatisticas,
                                     y1_lab: y1_res,
                                     y2_lab: y2_res,
                                     y3_lab: y3_res}
                            
                                confidence_interval = pd.DataFrame(data = d)
                            
                            else:
                                d = {'p-valor do ANOVA: probabilidade de as médias serem iguais': [p_val]}
                                     
                                confidence_interval = pd.DataFrame(data = d)
                            
                            if (plot_boxplot == True):
                            
                                #construir box plot
                                fig, ax = plt.subplots()
                                
                                ax.boxplot([y1, y2, y3], labels = [y1_lab, y2_lab, y3_lab])
                                ax.set_title('Box plot')
                                ax.set(ylabel='Variables')
                                
                                if not (reference_line is None):
                                    ax.axhline(reference_line, color = 'red', linestyle = 'dashed', label = 'Reference')
                                
                                #Criar linha horizontal tenue, que apenas guie os olhos se distrair
                                ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey', alpha=0.5)
                                ax.grid(True) #mude para False, caso não deseje ver as linhas de grade      
                                ax.legend()
                                plt.show()
                    
                            return confidence_interval
                        
                    if (n_factors == 4):
                            #temos 4 subgrupos
                           
                            y1, y2, y3, y4 = np.split(ary = y, indices_or_sections = indices_of_factors, axis=0)
                            y1 = y1.reset_index(drop=True)
                            y2 = y2.reset_index(drop=True)
                            y3 = y3.reset_index(drop=True)
                            y4 = y4.reset_index(drop=True)
                            n1 = len(y1)
                            n2 = len(y2)
                            n3 = len(y3)
                            n4 = len(y4)
                            
                            anova = stats.f_oneway(y1, y2, y3, y4)
                            p_val = anova[1]
                            #p-valor do anova
                            
                            if not (factor_labels is None):
                                y1_lab = factor_labels[0]
                                y2_lab = factor_labels[1]
                                y3_lab = factor_labels[2]
                                y4_lab = factor_labels[3]
                            
                            else:
                                factors1, factors2, factors3, factors4 = np.split(ary = factors, indices_or_sections = indices_of_factors, axis=0)
                                factors1 = factors1.reset_index(drop=True)
                                factors2 = factors2.reset_index(drop=True)
                                factors3 = factors3.reset_index(drop=True)
                                factors4 = factors4.reset_index(drop=True)
                                y1_lab = factors1[0]
                                y2_lab = factors2[0]
                                y3_lab = factors3[0]
                                y4_lab = factors4[0]
                                #procedimento que consome memoria extra: pode
                                #ser evitando fornecendo-se a lista pronta
                            
                            if (calculate_confidence_interval == True):
                                
                                nivel = confidence #nao perder o valor
                                nivel = nivel*100 #registrar em porcentagem
                                confidence = 1 - confidence
                                #alfa da t student bicaudal
                                confidence = confidence/2
                                #alfa/2
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom1 = n1-1
                                xbar1 = np.average(y1) #media
                                s1 = np.std(y1) #desv-pad
                                t1 = stats.t.ppf((1-confidence), degree_freedom1)
                                        
                                #intervalo de confianca da media:
                                lim_inf1 = xbar1 - t1*s1/(np.sqrt(n1))
                                lim_sup1 = xbar1 + t1*s1/(np.sqrt(n1))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom2 = n2-1
                                xbar2 = np.average(y2) #media
                                s2 = np.std(y2) #desv-pad
                                t2 = stats.t.ppf((1-confidence), degree_freedom2)
                                        
                                #intervalo de confianca da media:
                                lim_inf2 = xbar2 - t2*s2/(np.sqrt(n2))
                                lim_sup2 = xbar2 + t2*s2/(np.sqrt(n2))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom3 = n3-1
                                xbar3 = np.average(y3) #media
                                s3 = np.std(y3) #desv-pad
                                t3 = stats.t.ppf((1-confidence), degree_freedom3)
                                        
                                #intervalo de confianca da media:
                                lim_inf3 = xbar3 - t3*s3/(np.sqrt(n3))
                                lim_sup3 = xbar3 + t3*s3/(np.sqrt(n3))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom4 = n4-1
                                xbar4 = np.average(y4) #media
                                s4 = np.std(y4) #desv-pad
                                t4 = stats.t.ppf((1-confidence), degree_freedom4)
                                        
                                #intervalo de confianca da media:
                                lim_inf4 = xbar4 - t4*s4/(np.sqrt(n4))
                                lim_sup4 = xbar4 + t4*s4/(np.sqrt(n4))
                                
                                estatisticas = []
                                y1_res = []
                                y2_res = []
                                y3_res = []
                                y4_res = []
                                
                                string1 = "Nível de confiança (em porcentagem) = %.2f" %(nivel)
                                estatisticas.append(string1)
                                y1_res.append(None)
                                y2_res.append(None)
                                y3_res.append(None)
                                y4_res.append(None)
                                string2 = "Limite inferior do intervalo de confiança da média"
                                estatisticas.append(string2)
                                y1_res.append(lim_inf1)
                                y2_res.append(lim_inf2)
                                y3_res.append(lim_inf3)
                                y4_res.append(lim_inf4)
                                texto_extra = "Média amostral: centro do intervalo de confiança"
                                estatisticas.append(texto_extra)
                                y1_res.append(xbar1)
                                y2_res.append(xbar2)
                                y3_res.append(xbar3)
                                y4_res.append(xbar4)
                                string3 = 'Limite superior do intervalo de confiança da média'
                                estatisticas.append(string3)
                                y1_res.append(lim_sup1)
                                y2_res.append(lim_sup2)
                                y3_res.append(lim_sup3)
                                y4_res.append(lim_sup4)
                                string4 = "p-valor do ANOVA:\nProbabilidade de as médias serem iguais"
                                estatisticas.append(string4)
                                y1_res.append(p_val)
                                y2_res.append(p_val)
                                y3_res.append(p_val)
                                y4_res.append(p_val)
                                string5 = "Probabilidade de as médias serem iguais (em %)"
                                estatisticas.append(string5)
                                y1_res.append(p_val*100)
                                y2_res.append(p_val*100)
                                y3_res.append(p_val*100)
                                y4_res.append(p_val*100)
                                
                                if not (reference_line is None):
                                    string6 = "Valor de referência: %.2f" %(reference_line)
                                    estatisticas.append(string6)
                                    if (lim_inf1 < reference_line) and (reference_line < lim_sup1):
                                        y1_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y1_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf2 < reference_line) and (reference_line < lim_sup2):
                                        y2_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y2_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf3 < reference_line) and (reference_line < lim_sup3):
                                        y3_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y3_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf4 < reference_line) and (reference_line < lim_sup4):
                                        y4_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y4_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                
                                d = {'Valor': estatisticas,
                                     y1_lab: y1_res,
                                     y2_lab: y2_res,
                                     y3_lab: y3_res,
                                     y4_lab: y4_res}
                            
                                confidence_interval = pd.DataFrame(data = d)
                            
                            else:
                                d = {'p-valor do ANOVA: probabilidade de as médias serem iguais': [p_val]}
                                     
                                confidence_interval = pd.DataFrame(data = d)
                            
                            if (plot_boxplot == True):
                            
                                #construir box plot
                                fig, ax = plt.subplots()
                                
                                ax.boxplot([y1, y2, y3, y4], labels = [y1_lab, y2_lab, y3_lab, y4_lab])
                                ax.set_title('Box plot')
                                ax.set(ylabel='Variables')
                                
                                if not (reference_line is None):
                                    ax.axhline(reference_line, color = 'red', linestyle = 'dashed', label = 'Reference')
                                
                                #Criar linha horizontal tenue, que apenas guie os olhos se distrair
                                ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey', alpha=0.5)
                                ax.grid(True) #mude para False, caso não deseje ver as linhas de grade      
                                ax.legend()
                                plt.show()
                    
                            return confidence_interval
                        
                    if (n_factors == 5):
                            #temos 5 subgrupos
                           
                            y1, y2, y3, y4, y5 = np.split(ary = y, indices_or_sections = indices_of_factors, axis=0)
                            y1 = y1.reset_index(drop=True)
                            y2 = y2.reset_index(drop=True)
                            y3 = y3.reset_index(drop=True)
                            y4 = y4.reset_index(drop=True)
                            y5 = y5.reset_index(drop=True)
                            n1 = len(y1)
                            n2 = len(y2)
                            n3 = len(y3)
                            n4 = len(y4)
                            n5 = len(y5)
                            
                            anova = stats.f_oneway(y1, y2, y3, y4, y5)
                            p_val = anova[1]
                            #p-valor do anova
                            
                            if not (factor_labels is None):
                                y1_lab = factor_labels[0]
                                y2_lab = factor_labels[1]
                                y3_lab = factor_labels[2]
                                y4_lab = factor_labels[3]
                                y5_lab = factor_labels[4]
                            
                            else:
                                factors1, factors2, factors3, factors4, factors5 = np.split(ary = factors, indices_or_sections = indices_of_factors, axis=0)
                                factors1 = factors1.reset_index(drop=True)
                                factors2 = factors2.reset_index(drop=True)
                                factors3 = factors3.reset_index(drop=True)
                                factors4 = factors4.reset_index(drop=True)
                                factors5 = factors5.reset_index(drop=True)
                                y1_lab = factors1[0]
                                y2_lab = factors2[0]
                                y3_lab = factors3[0]
                                y4_lab = factors4[0]
                                y5_lab = factors5[0]
                                #procedimento que consome memoria extra: pode
                                #ser evitando fornecendo-se a lista pronta
                            
                            if (calculate_confidence_interval == True):
                                
                                nivel = confidence #nao perder o valor
                                nivel = nivel*100 #registrar em porcentagem
                                confidence = 1 - confidence
                                #alfa da t student bicaudal
                                confidence = confidence/2
                                #alfa/2
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom1 = n1-1
                                xbar1 = np.average(y1) #media
                                s1 = np.std(y1) #desv-pad
                                t1 = stats.t.ppf((1-confidence), degree_freedom1)
                                        
                                #intervalo de confianca da media:
                                lim_inf1 = xbar1 - t1*s1/(np.sqrt(n1))
                                lim_sup1 = xbar1 + t1*s1/(np.sqrt(n1))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom2 = n2-1
                                xbar2 = np.average(y2) #media
                                s2 = np.std(y2) #desv-pad
                                t2 = stats.t.ppf((1-confidence), degree_freedom2)
                                        
                                #intervalo de confianca da media:
                                lim_inf2 = xbar2 - t2*s2/(np.sqrt(n2))
                                lim_sup2 = xbar2 + t2*s2/(np.sqrt(n2))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom3 = n3-1
                                xbar3 = np.average(y3) #media
                                s3 = np.std(y3) #desv-pad
                                t3 = stats.t.ppf((1-confidence), degree_freedom3)
                                        
                                #intervalo de confianca da media:
                                lim_inf3 = xbar3 - t3*s3/(np.sqrt(n3))
                                lim_sup3 = xbar3 + t3*s3/(np.sqrt(n3))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom4 = n4-1
                                xbar4 = np.average(y4) #media
                                s4 = np.std(y4) #desv-pad
                                t4 = stats.t.ppf((1-confidence), degree_freedom4)
                                        
                                #intervalo de confianca da media:
                                lim_inf4 = xbar4 - t4*s4/(np.sqrt(n4))
                                lim_sup4 = xbar4 + t4*s4/(np.sqrt(n4))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom5 = n5-1
                                xbar5 = np.average(y5) #media
                                s5 = np.std(y5) #desv-pad
                                t5 = stats.t.ppf((1-confidence), degree_freedom5)
                                        
                                #intervalo de confianca da media:
                                lim_inf5 = xbar5 - t5*s5/(np.sqrt(n5))
                                lim_sup5 = xbar5 + t5*s5/(np.sqrt(n5))
                                
                                estatisticas = []
                                y1_res = []
                                y2_res = []
                                y3_res = []
                                y4_res = []
                                y5_res = []
                                
                                string1 = "Nível de confiança (em porcentagem) = %.2f" %(nivel)
                                estatisticas.append(string1)
                                y1_res.append(None)
                                y2_res.append(None)
                                y3_res.append(None)
                                y4_res.append(None)
                                y5_res.append(None)
                                string2 = "Limite inferior do intervalo de confiança da média"
                                estatisticas.append(string2)
                                y1_res.append(lim_inf1)
                                y2_res.append(lim_inf2)
                                y3_res.append(lim_inf3)
                                y4_res.append(lim_inf4)
                                y5_res.append(lim_inf5)
                                texto_extra = "Média amostral: centro do intervalo de confiança"
                                estatisticas.append(texto_extra)
                                y1_res.append(xbar1)
                                y2_res.append(xbar2)
                                y3_res.append(xbar3)
                                y4_res.append(xbar4)
                                y5_res.append(xbar5)
                                string3 = 'Limite superior do intervalo de confiança da média'
                                estatisticas.append(string3)
                                y1_res.append(lim_sup1)
                                y2_res.append(lim_sup2)
                                y3_res.append(lim_sup3)
                                y4_res.append(lim_sup4)
                                y5_res.append(lim_sup5)
                                string4 = "p-valor do ANOVA:\nProbabilidade de as médias serem iguais"
                                estatisticas.append(string4)
                                y1_res.append(p_val)
                                y2_res.append(p_val)
                                y3_res.append(p_val)
                                y4_res.append(p_val)
                                y5_res.append(p_val)
                                string5 = "Probabilidade de as médias serem iguais (em %)"
                                estatisticas.append(string5)
                                y1_res.append(p_val*100)
                                y2_res.append(p_val*100)
                                y3_res.append(p_val*100)
                                y4_res.append(p_val*100)
                                y5_res.append(p_val*100)
                                
                                if not (reference_line is None):
                                    string6 = "Valor de referência: %.2f" %(reference_line)
                                    estatisticas.append(string6)
                                    if (lim_inf1 < reference_line) and (reference_line < lim_sup1):
                                        y1_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y1_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf2 < reference_line) and (reference_line < lim_sup2):
                                        y2_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y2_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf3 < reference_line) and (reference_line < lim_sup3):
                                        y3_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y3_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf4 < reference_line) and (reference_line < lim_sup4):
                                        y4_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y4_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf5 < reference_line) and (reference_line < lim_sup5):
                                        y5_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y5_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                
                                d = {'Valor': estatisticas,
                                     y1_lab: y1_res,
                                     y2_lab: y2_res,
                                     y3_lab: y3_res,
                                     y4_lab: y4_res,
                                     y5_lab: y5_res}
                            
                                confidence_interval = pd.DataFrame(data = d)
                            
                            else:
                                d = {'p-valor do ANOVA: probabilidade de as médias serem iguais': [p_val]}
                                     
                                confidence_interval = pd.DataFrame(data = d)
                            
                            if (plot_boxplot == True):
                            
                                #construir box plot
                                fig, ax = plt.subplots()
                                
                                ax.boxplot([y1, y2, y3, y4, y5], labels = [y1_lab, y2_lab, y3_lab, y4_lab, y5_lab])
                                ax.set_title('Box plot')
                                ax.set(ylabel='Variables')
                                
                                if not (reference_line is None):
                                    ax.axhline(reference_line, color = 'red', linestyle = 'dashed', label = 'Reference')
                                
                                #Criar linha horizontal tenue, que apenas guie os olhos se distrair
                                ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey', alpha=0.5)
                                ax.grid(True) #mude para False, caso não deseje ver as linhas de grade      
                                ax.legend()
                                plt.show()
                    
                            return confidence_interval

                    if (n_factors == 6):
                            #temos 6 subgrupos
                           
                            y1, y2, y3, y4, y5, y6 = np.split(ary = y, indices_or_sections = indices_of_factors, axis=0)
                            y1 = y1.reset_index(drop=True)
                            y2 = y2.reset_index(drop=True)
                            y3 = y3.reset_index(drop=True)
                            y4 = y4.reset_index(drop=True)
                            y5 = y5.reset_index(drop=True)
                            y6 = y6.reset_index(drop=True)
                            n1 = len(y1)
                            n2 = len(y2)
                            n3 = len(y3)
                            n4 = len(y4)
                            n5 = len(y5)
                            n6 = len(y6)
                            
                            anova = stats.f_oneway(y1, y2, y3, y4, y5, y6)
                            p_val = anova[1]
                            #p-valor do anova
                            
                            if not (factor_labels is None):
                                y1_lab = factor_labels[0]
                                y2_lab = factor_labels[1]
                                y3_lab = factor_labels[2]
                                y4_lab = factor_labels[3]
                                y5_lab = factor_labels[4]
                                y6_lab = factor_labels[5]
                            
                            else:
                                factors1, factors2, factors3, factors4, factors5, factors6 = np.split(ary = factors, indices_or_sections = indices_of_factors, axis=0)
                                factors1 = factors1.reset_index(drop=True)
                                factors2 = factors2.reset_index(drop=True)
                                factors3 = factors3.reset_index(drop=True)
                                factors4 = factors4.reset_index(drop=True)
                                factors5 = factors5.reset_index(drop=True)
                                factors6 = factors6.reset_index(drop=True)
                                y1_lab = factors1[0]
                                y2_lab = factors2[0]
                                y3_lab = factors3[0]
                                y4_lab = factors4[0]
                                y5_lab = factors5[0]
                                y6_lab = factors6[0]
                                #procedimento que consome memoria extra: pode
                                #ser evitando fornecendo-se a lista pronta
                            
                            if (calculate_confidence_interval == True):
                                
                                nivel = confidence #nao perder o valor
                                nivel = nivel*100 #registrar em porcentagem
                                confidence = 1 - confidence
                                #alfa da t student bicaudal
                                confidence = confidence/2
                                #alfa/2
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom1 = n1-1
                                xbar1 = np.average(y1) #media
                                s1 = np.std(y1) #desv-pad
                                t1 = stats.t.ppf((1-confidence), degree_freedom1)
                                        
                                #intervalo de confianca da media:
                                lim_inf1 = xbar1 - t1*s1/(np.sqrt(n1))
                                lim_sup1 = xbar1 + t1*s1/(np.sqrt(n1))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom2 = n2-1
                                xbar2 = np.average(y2) #media
                                s2 = np.std(y2) #desv-pad
                                t2 = stats.t.ppf((1-confidence), degree_freedom2)
                                        
                                #intervalo de confianca da media:
                                lim_inf2 = xbar2 - t2*s2/(np.sqrt(n2))
                                lim_sup2 = xbar2 + t2*s2/(np.sqrt(n2))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom3 = n3-1
                                xbar3 = np.average(y3) #media
                                s3 = np.std(y3) #desv-pad
                                t3 = stats.t.ppf((1-confidence), degree_freedom3)
                                        
                                #intervalo de confianca da media:
                                lim_inf3 = xbar3 - t3*s3/(np.sqrt(n3))
                                lim_sup3 = xbar3 + t3*s3/(np.sqrt(n3))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom4 = n4-1
                                xbar4 = np.average(y4) #media
                                s4 = np.std(y4) #desv-pad
                                t4 = stats.t.ppf((1-confidence), degree_freedom4)
                                        
                                #intervalo de confianca da media:
                                lim_inf4 = xbar4 - t4*s4/(np.sqrt(n4))
                                lim_sup4 = xbar4 + t4*s4/(np.sqrt(n4))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom5 = n5-1
                                xbar5 = np.average(y5) #media
                                s5 = np.std(y5) #desv-pad
                                t5 = stats.t.ppf((1-confidence), degree_freedom5)
                                        
                                #intervalo de confianca da media:
                                lim_inf5 = xbar5 - t5*s5/(np.sqrt(n5))
                                lim_sup5 = xbar5 + t5*s5/(np.sqrt(n5))
                                
                                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                                degree_freedom6 = n6-1
                                xbar6 = np.average(y6) #media
                                s6 = np.std(y6) #desv-pad
                                t6 = stats.t.ppf((1-confidence), degree_freedom6)
                                        
                                #intervalo de confianca da media:
                                lim_inf6 = xbar6 - t6*s6/(np.sqrt(n6))
                                lim_sup6 = xbar6 + t6*s6/(np.sqrt(n6))
                                
                                estatisticas = []
                                y1_res = []
                                y2_res = []
                                y3_res = []
                                y4_res = []
                                y5_res = []
                                y6_res = []
                                
                                string1 = "Nível de confiança (em porcentagem) = %.2f" %(nivel)
                                estatisticas.append(string1)
                                y1_res.append(None)
                                y2_res.append(None)
                                y3_res.append(None)
                                y4_res.append(None)
                                y5_res.append(None)
                                y6_res.append(None)
                                string2 = "Limite inferior do intervalo de confiança da média"
                                estatisticas.append(string2)
                                y1_res.append(lim_inf1)
                                y2_res.append(lim_inf2)
                                y3_res.append(lim_inf3)
                                y4_res.append(lim_inf4)
                                y5_res.append(lim_inf5)
                                y6_res.append(lim_inf6)
                                texto_extra = "Média amostral: centro do intervalo de confiança"
                                estatisticas.append(texto_extra)
                                y1_res.append(xbar1)
                                y2_res.append(xbar2)
                                y3_res.append(xbar3)
                                y4_res.append(xbar4)
                                y5_res.append(xbar5)
                                y6_res.append(xbar6)
                                string3 = 'Limite superior do intervalo de confiança da média'
                                estatisticas.append(string3)
                                y1_res.append(lim_sup1)
                                y2_res.append(lim_sup2)
                                y3_res.append(lim_sup3)
                                y4_res.append(lim_sup4)
                                y5_res.append(lim_sup5)
                                y6_res.append(lim_sup6)
                                string4 = "p-valor do ANOVA:\nProbabilidade de as médias serem iguais"
                                estatisticas.append(string4)
                                y1_res.append(p_val)
                                y2_res.append(p_val)
                                y3_res.append(p_val)
                                y4_res.append(p_val)
                                y5_res.append(p_val)
                                y6_res.append(p_val)
                                string5 = "Probabilidade de as médias serem iguais (em %)"
                                estatisticas.append(string5)
                                y1_res.append(p_val*100)
                                y2_res.append(p_val*100)
                                y3_res.append(p_val*100)
                                y4_res.append(p_val*100)
                                y5_res.append(p_val*100)
                                y6_res.append(p_val*100)
                                
                                if not (reference_line is None):
                                    string6 = "Valor de referência: %.2f" %(reference_line)
                                    estatisticas.append(string6)
                                    if (lim_inf1 < reference_line) and (reference_line < lim_sup1):
                                        y1_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y1_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf2 < reference_line) and (reference_line < lim_sup2):
                                        y2_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y2_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf3 < reference_line) and (reference_line < lim_sup3):
                                        y3_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y3_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf4 < reference_line) and (reference_line < lim_sup4):
                                        y4_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y4_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf5 < reference_line) and (reference_line < lim_sup5):
                                        y5_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y5_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                                    if (lim_inf6 < reference_line) and (reference_line < lim_sup6):
                                        y6_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                                    else:
                                        y6_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                
                                d = {'Valor': estatisticas,
                                     y1_lab: y1_res,
                                     y2_lab: y2_res,
                                     y3_lab: y3_res,
                                     y4_lab: y4_res,
                                     y5_lab: y5_res,
                                     y6_lab: y6_res}
                            
                                confidence_interval = pd.DataFrame(data = d)
                            
                            else:
                                d = {'p-valor do ANOVA: probabilidade de as médias serem iguais': [p_val]}
                                     
                                confidence_interval = pd.DataFrame(data = d)
                            
                            if (plot_boxplot == True):
                            
                                #construir box plot
                                fig, ax = plt.subplots()
                                
                                ax.boxplot([y1, y2, y3, y4, y5, y6], labels = [y1_lab, y2_lab, y3_lab, y4_lab, y5_lab, y6_lab])
                                ax.set_title('Box plot')
                                ax.set(ylabel='Variables')
                                
                                if not (reference_line is None):
                                    ax.axhline(reference_line, color = 'red', linestyle = 'dashed', label = 'Reference')
                                
                                #Criar linha horizontal tenue, que apenas guie os olhos se distrair
                                ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey', alpha=0.5)
                                ax.grid(True) #mude para False, caso não deseje ver as linhas de grade      
                                ax.legend()
                                plt.show()
                    
                            return confidence_interval

# CÉLULA 8: Variavel em mesma coluna

### ATENÇÃO: NÃO EXECUTE ESTA CÉLULA - Ela contém apenas texto explicativo

Função para Análise de Variância (ANOVA), cálculo de intervalos de confiança das médias, e construção de gráficos Box Plot

INSTRUÇÕES:    
    
Esta função deve ser usada quando os dados estão em uma mesma coluna, e uma segunda coluna é usada para indicar tipos/fatores/grupos.

##### ATENÇÃO: AS AMOSTRAS DEVEM TER O MESMO NÚMERO DE DADOS PARA QUE O p-valor SEJA CALCULADO. 

Caso não tenham, remova manualmente linhas da amostra maior para igualar o total de valores. Caso contrário, os intervalos de confiança das médias serão obtidos sem o cálculo da probabilidade de as médias serem iguais.

Compare um máximo de 6 fatores simultâneos ou expanda o código, copiando e replicando os índices. Mude o condicional de que não pode ser maior que 6.

    ATENÇÃO: Esta função retorna como saída um dataframe. Logo, não deixe de especificar um dataframe para receber a saída. 
    
Exemplo:

    df = anova_boxplot_same_column(dataframe, col_y_name = "Y", col_factor_name = "Factors", confidence = 0.95, factor_labels = ["A", "B", "C"], reference_line = 7.85,  plot_boxplot = True, calculate_confidence_interval = True)

PARÂMETROS DE ENTRADA:
    
    anova_boxplot_same_column(dataframe, col_y_name, col_factor_name, confidence = 0.95, factor_labels = None, reference_line = None,  plot_boxplot = True, calculate_confidence_interval = True)

1) dataframe: dataframe no qual os dados se encontram

2) col_y_name: nome da coluna (variável) na qual se encontram os dados y que serão analisados com o ANOVA. Deve vir entre aspas e exatamente igual à forma como está escrito. 

        Exemplo: col_y_name = "Y"

3) col_factor_name: nome da coluna (variável) na qual se encontram os fatores/grupos segundo os quais os dados y estão dividos. 

##### O objetivo é VERIFICAR SE A MUDANÇA DE FATOR CAUSA MUDANÇA NA MÉDIA, por meio da análise de variância (ANOVA). 

Basicamente, o ANOVA avalia a probabilidade de a soma das variâncias resultar na variância observada. Deve vir entre aspas e exatamente igual à forma como está escrito. 

        Exemplo: col_factor_name = "Grupo"

4) confidence: nível de confiança para o cálculo do intervalo da média (ou seja, a probabilidade de a média se encontrar neste intervalo = confidence).
    
        Mantenha confidence = 0.95 para 95% de confiança
        
        - Caso deseje reduzir, basta alterar o valor: confidence = 0.90 fornece 90% de confiança

5) factor_labels: são os rótulos/nomes de cada um dos fatores. Deve ser fornecido como uma lista, com cada elemento separado por vírgula. Caso os fatores sejam numéricos, eles não precisam ser declarados entre aspas. Caso sejam letras ou palavras (strings), devem vir entre aspas. O número de elementos da lista deve ser EXATAMENTE IGUAL ao número de fatores. Exemplos:
    
        factor_labels = ["A", "B", "C"]
        factor_labels = ["Verde", "Azul", "Amarelo", "Preto"]
        factor_labels = ["Companhia A", "Companhia B"]
        factor_labels = [0, 1, 2, 3, 4]
    
Caso não deseje colocar labels, mantenha factor_labels = None. Isto fará o código identificar na coluna dos fatores quais os diferentes labels, resultando em consumo extra de memória.
    
6) reference_line: este parâmetro colocará uma linha de referência horizontal no box plot, permitindo a visualização se algum dos intervalos avaliados engloba ou não esta linha de referência (evidenciando se existe ou não elementos que permitam dizer que a média é ou não diferente daquele valor).

        Declare como um valor numérico ou mantenha reference_line = None, caso não deseje incluir linhas. 
        
Caso haja linha de referência e cálculo do intervalo de confiança, o dataframe de saída informará se o valor de referência pertence ou não aos intervalos de confiança das médias. 

        Exemplo: reference_line = 10.25 incluirá uma reta horizontal na altura Y = 10.25 do boxplot. 

Note que o número não deve vir entre aspas, e que o separador decimal é o ponto.

7) plot_boxplot: mantenha plot_boxplot = True, caso deseje que o box plot seja gerado, ou mude para plot_boxplot = False, caso não deseje gerar o gráfico.

Note que apenas os valores True e False são aceitos, e eles não devem vir entre aspas.

8) calculate_confidence_interval: mantenha calculate_confidence_interval = True, caso deseje calcular o intervalo de confiança da média. 

###### Neste caso, além do p-valor do ANOVA (probabilidade de as médias serem iguais), o dataframe de saída mostrará os limites inferior e superior dos intervalos de confiança para o nível de confiança especificado em 'confidence'. 

Caso não deseje calcular os intervalos, mude para calculate_confidence_interval = False.

Note que apenas os valores True e False são aceitos, e eles não devem vir entre aspas.

###### t-Student
seja xbar a média de n medições, cujo desvio-padrão amostral é s

seja t(confidence; gl) a distribuição t-Student com confidence*100% de confiança e gl = n-1 graus de liberdade

t = (xbar +- mu)/(s/sqrt(n)), onde sqrt = square root = raiz quadrada

o intervalo de confiança da média será dado por:

xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)

In [ ]:
## CÉLULA 9: ANÁLISE DE VARIÂNCIA, CÁLCULO DOS INTERVALOS DE CONFIANÇA E CONSTRUÇÃO DO BOXPLOT PARA SEUS DADOS

#DEFINA O NOME DA COLUNA COM AS VARIÁVEIS Y comparadas. Basta substituir o valor entre aspas pelo nome da coluna onde está Y:
nome_y = "SUBSTITUA ESTE TEXTO ENTRE ASPAS PELO NOME DA VARIÁVEL Y (Título da coluna)"

#DEFINA O NOME DA COLUNA COM OS GRUPOS/TRATAMENTOS/FATORES. Basta substituir o valor entre aspas pelo nome da coluna 
#na qual são identificados os fatores:
nome_fator = "SUBSTITUA ESTE TEXTO ENTRE ASPAS PELO NOME DA COLUNA DOS GRUPOS/TRATAMENTOS/FATORES (Título da coluna)"

confidence_interval = anova_boxplot_same_column(dataframe = dataset, col_y_name = nome_y, col_factor_name = nome_fator, confidence = 0.95, factor_labels = None, reference_line = None,  plot_boxplot = True, calculate_confidence_interval = True)

"""
Substitua os demais campos de acordo com as instruções dadas na CÉLULA 8.
"""

confidence_interval

"""VOCÊ DESEJA EXPORTAR OS DADOS?
Caso deseje exportar os dados, copie a seguinte linha para o espaço não-vermelho após as aspas. Substitua o endereço pela
pasta onde você deseja salvar seu arquivo. Substitua int_confianca pelo nome que deseja para seu arquivo. Mantenha
a extensão csv

confidence_interval.to_csv(r"D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 3 - ANN classification\ANN.Class - 1 - user satisfaction\int_confianca.csv", index = False)

"""

In [ ]:
## CÉLULA 10: Função para análise de variância (ANOVA) - Verificar se as médias dos resultados correspondentes a diferentes
# tratamentos são ou não diferentes. 
# Neste caso, a função analisa a situação na qual os dados correspondentes a tratamentos diferentes estão em colunas separadas
# Não existe uma coluna que identifique tags/marcadores
# Colunas diferentes CORRESPONDEM a tratamentos diferentes.

#Execute esta CÉLULA para carregar a função

def anova_boxplot_separate_columns(dataframe, col_y1_name, col_y2_name, confidence = 0.95, col_y3_name = None, col_y4_name = None, col_y5_name = None, col_y6_name = None, reference_line = None,  plot_boxplot = True, calculate_confidence_interval = True):

    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    from scipy import stats
    
    if (col_y2_name is None):
        print("\nError! Only one group! You must compare at least two groups with the ANOVA!\n")
        print("If you only want to check the confidence interval of the mean value, use the function 'anova_boxplot_same_column', which allows you to only calculate the confidence interval, without performing the analysis of variance.\n")
        
    y1_lab = col_y1_name
    y1 = dataframe[y1_lab]
    n1 = len(y1)
    xbar1 = y1.mean()
    s1 = y1.std()
    y2_lab = col_y2_name
    y2 = dataframe[y2_lab]
    n2 = len(y2)
    xbar2 = y2.mean()
    s2 = y2.std()
    
    if not (col_y3_name is None):
        y3_lab = col_y3_name
        y3 = dataframe[y3_lab]
        n3 = len(y3)
        xbar3 = y3.mean()
        s3 = y3.std()
        
        if not (col_y4_name is None):
            y4_lab = col_y4_name
            y4 = dataframe[y4_lab]
            n4 = len(y4)
            xbar4 = y4.mean()
            s4 = y4.std()
            
            if not (col_y5_name is None):
                y5_lab = col_y5_name
                y5 = dataframe[y5_lab]
                n5 = len(y5)
                xbar5 = y5.mean()
                s5 = y5.std()
                
                if not (col_y6_name is None):
                    y6_lab = col_y6_name
                    y6 = dataframe[y6_lab]
                    n6 = len(y6)
                    xbar6 = y6.mean()
                    s6 = y6.std()
                    
                    #caso os 6 fatores estejam presentes, calcula o ANOVA
                    #para os 6:
                    anova = stats.f_oneway(y1, y2, y3, y4, y5, y6)
                    #p-valor do anova:
                    p_val = anova[1]
                
                else:
                    #caso em que existem 5 fatores:
                    anova = stats.f_oneway(y1, y2, y3, y4, y5)
                    #p-valor do anova:
                    p_val = anova[1]
            
            else:
                #caso em que existem 4 fatores:
                anova = stats.f_oneway(y1, y2, y3, y4)
                #p-valor do anova:
                p_val = anova[1]
        
        else:
            #caso em que existem 3 fatores:
            anova = stats.f_oneway(y1, y2, y3)
            #p-valor do anova:
            p_val = anova[1]
    
    else:
        #caso em que so 2 fatores estao presentes:
        anova = stats.f_oneway(y1, y2)
        #p-valor do anova:
        p_val = anova[1]
    
    if (calculate_confidence_interval == True):
        
        estatisticas = []
        y1_res = []
        y2_res = []
        
        nivel = confidence #nao perder o valor
        nivel = nivel*100 #registrar em porcentagem
        confidence = 1 - confidence
        #alfa da t student bicaudal
        confidence = confidence/2
        #alfa/2
                                
        #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
        degree_freedom1 = n1-1
        t1 = stats.t.ppf((1-confidence), degree_freedom1)
        #intervalo de confianca da media:
        lim_inf1 = xbar1 - t1*s1/(np.sqrt(n1))
        lim_sup1 = xbar1 + t1*s1/(np.sqrt(n1))
                                
        #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
        degree_freedom2 = n2-1
        t2 = stats.t.ppf((1-confidence), degree_freedom2)
        #intervalo de confianca da media:
        lim_inf2 = xbar2 - t2*s2/(np.sqrt(n2))
        lim_sup2 = xbar2 + t2*s2/(np.sqrt(n2))
        
        string1 = "Nível de confiança (em porcentagem) = %.2f" %(nivel)
        estatisticas.append(string1)
        y1_res.append(None)
        y2_res.append(None)
        string2 = "Limite inferior do intervalo de confiança da média"
        estatisticas.append(string2)
        y1_res.append(lim_inf1)
        y2_res.append(lim_inf2)
        texto_extra = "Média amostral: centro do intervalo de confiança"
        estatisticas.append(texto_extra)
        y1_res.append(xbar1)
        y2_res.append(xbar2)
        string3 = 'Limite superior do intervalo de confiança da média'
        estatisticas.append(string3)
        y1_res.append(lim_sup1)
        y2_res.append(lim_sup2)
        string4 = "p-valor do ANOVA:\nProbabilidade de as médias serem iguais"
        estatisticas.append(string4)
        y1_res.append(p_val)
        y2_res.append(p_val)
        string5 = "Probabilidade de as médias serem iguais (em %)"
        estatisticas.append(string5)
        y1_res.append(p_val*100)
        y2_res.append(p_val*100)
        
        if not (reference_line is None):
            string6 = "Valor de referência: %.2f" %(reference_line)
            estatisticas.append(string6)
            if (lim_inf1 < reference_line) and (reference_line < lim_sup1):
                y1_res.append("Valor de referência pertence\nao intervalo de confiança da média")
            else:
                y1_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
            
            if (lim_inf2 < reference_line) and (reference_line < lim_sup2):
                y2_res.append("Valor de referência pertence\nao intervalo de confiança da média")
            else:
                y2_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
        
        if not (col_y3_name is None):
            y3_res = []
            #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
            degree_freedom3 = n3-1
            t3 = stats.t.ppf((1-confidence), degree_freedom3)
            #intervalo de confianca da media:
            lim_inf3 = xbar3 - t3*s3/(np.sqrt(n3))
            lim_sup3 = xbar3 + t3*s3/(np.sqrt(n3))
            y3_res.append(None)
            y3_res.append(lim_inf3)
            y3_res.append(xbar3)
            y3_res.append(lim_sup3)
            y3_res.append(p_val)
            y3_res.append(p_val*100)
            
            if not (reference_line is None):
                if (lim_inf3 < reference_line) and (reference_line < lim_sup3):
                    y3_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                else:
                    y3_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
            if not (col_y4_name is None):
                y4_res = []
                #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                degree_freedom4 = n4-1
                t4 = stats.t.ppf((1-confidence), degree_freedom4)
                #intervalo de confianca da media:
                lim_inf4 = xbar4 - t4*s4/(np.sqrt(n4))
                lim_sup4 = xbar4 + t4*s4/(np.sqrt(n4))
                y4_res.append(None)
                y4_res.append(lim_inf4)
                y4_res.append(xbar4)
                y4_res.append(lim_sup4)
                y4_res.append(p_val)
                y4_res.append(p_val*100)
                
                if not (reference_line is None):
                    if (lim_inf4 < reference_line) and (reference_line < lim_sup4):
                        y4_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                    else:
                        y4_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                if not (col_y5_name is None):
                    y5_res = []
                    #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                    degree_freedom5 = n5-1
                    t5 = stats.t.ppf((1-confidence), degree_freedom5)
                    #intervalo de confianca da media:
                    lim_inf5 = xbar5 - t5*s5/(np.sqrt(n5))
                    lim_sup5 = xbar5 + t5*s5/(np.sqrt(n5))
                    y5_res.append(None)
                    y5_res.append(lim_inf5)
                    y5_res.append(xbar5)
                    y5_res.append(lim_sup5)
                    y5_res.append(p_val)
                    y5_res.append(p_val*100)
                    
                    if not (reference_line is None):
                        if (lim_inf5 < reference_line) and (reference_line < lim_sup5):
                            y5_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                        else:
                            y5_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                                    
                    if not (col_y6_name is None):
                        y6_res = []
                        #xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)
                        degree_freedom6 = n6-1
                        t6 = stats.t.ppf((1-confidence), degree_freedom6)
                        #intervalo de confianca da media:
                        lim_inf6 = xbar6 - t6*s6/(np.sqrt(n6))
                        lim_sup6 = xbar6 + t6*s6/(np.sqrt(n6))
                        y6_res.append(None)
                        y6_res.append(lim_inf6)
                        y6_res.append(xbar6)
                        y6_res.append(lim_sup6)                                
                        y6_res.append(p_val)
                        y6_res.append(p_val*100)
                        
                        if not (reference_line is None):
                            if (lim_inf6 < reference_line) and (reference_line < lim_sup6):
                                 y6_res.append("Valor de referência pertence\nao intervalo de confiança da média")
                            else:
                                y6_res.append("Valor de referência não pertence\nao intervalo de confiança da média")
                        
                        d = {'Valor': estatisticas,
                             y1_lab: y1_res,
                             y2_lab: y2_res,
                             y3_lab: y3_res,
                             y4_lab: y4_res,
                             y5_lab: y5_res,
                             y6_lab: y6_res}
                    
                    else:
                        #caso em que ha 5 fatores
                        d = {'Valor': estatisticas,
                             y1_lab: y1_res,
                             y2_lab: y2_res,
                             y3_lab: y3_res,
                             y4_lab: y4_res,
                             y5_lab: y5_res}
                
                else:
                    #caso em que ha 4 fatores
                    d = {'Valor': estatisticas,
                         y1_lab: y1_res,
                         y2_lab: y2_res,
                         y3_lab: y3_res,
                         y4_lab: y4_res}
            
            else:
                #caso em que ha 3 fatores
                d = {'Valor': estatisticas,
                     y1_lab: y1_res,
                     y2_lab: y2_res,
                     y3_lab: y3_res}
            
        else:
            #caso em que ha apenas 2 fatores
            d = {'Valor': estatisticas,
                 y1_lab: y1_res,
                 y2_lab: y2_res}
    
    else:
        #caso em que nao se deseja calcular os intervalos de confianca da media
        d = {'p-valor do ANOVA: probabilidade de as médias serem iguais': [p_val]}
                                     
    confidence_interval = pd.DataFrame(data = d)

    if (plot_boxplot == True):
                            
        #construir box plot
        fig, ax = plt.subplots()
           
        if not (col_y3_name is None):
        
            if not (col_y4_name is None):
            
                if not (col_y5_name is None):
                
                    if not (col_y6_name is None):
                        #caso os 6 fatores estejam presentes:
                        ax.boxplot([y1, y2, y3, y4, y5, y6], labels = [y1_lab, y2_lab, y3_lab, y4_lab, y5_lab, y6_lab])
                
                    else:
                        #caso em que existem 5 fatores:
                        ax.boxplot([y1, y2, y3, y4, y5], labels = [y1_lab, y2_lab, y3_lab, y4_lab, y5_lab])
                
                else:
                    #caso em que existem 4 fatores:
                    ax.boxplot([y1, y2, y3, y4], labels = [y1_lab, y2_lab, y3_lab, y4_lab])
        
            else:
                #caso em que existem 3 fatores:
                ax.boxplot([y1, y2, y3], labels = [y1_lab, y2_lab, y3_lab])
    
        else:
            #caso em que so 2 fatores estao presentes:
            ax.boxplot([y1, y2], labels = [y1_lab, y2_lab])
                     
        ax.set_title('Box plot')
        ax.set(ylabel='Variables')
                                
        if not (reference_line is None):
            ax.axhline(reference_line, color = 'red', linestyle = 'dashed', label = 'Reference')
                                
        #Criar linha horizontal tenue, que apenas guie os olhos se distrair
        ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey', alpha=0.5)
        ax.grid(True) #mude para False, caso não deseje ver as linhas de grade      
        ax.legend()
        plt.show()
                    
    return confidence_interval

# CÉLULA 11: Função para Análise de Variância (ANOVA), cálculo de intervalos de confiança das médias, e construção de gráficos Box Plot

### ATENÇÃO: NÃO EXECUTE ESTA CÉLULA - Ela contém apenas texto explicativo

INSTRUÇÕES:    
    
Esta função deve ser usada quando os dados estão em colunas separadas, de modo que cada coluna representa os dados obtidos para um grupo/tipo/fator.

Assim, o nome de cada coluna (variável) deve ser o nome do fator analisado.

Compare um máximo de 6 fatores simultâneos ou expanda o código, copiando e replicando os índices. Cuidado com as sequências de condicionais.

ATENÇÃO: Esta função retorna como saída um dataframe. Logo, não deixe de especificar um dataframe para receber a saída. 

Exemplo:

    df = anova_boxplot_separate_columns(dataframe, col_y1_name = "Y1", col_y2_name = "Y2", confidence = 0.95, col_y3_name = None, col_y4_name = None, col_y5_name = None, col_y6_name = None, reference_line = None,  plot_boxplot = True, calculate_confidence_interval = True)

PARÂMETROS DE ENTRADA:
    
    anova_boxplot_separate_columns(dataframe, col_y1_name, col_y2_name, confidence = 0.95, col_y3_name = None, col_y4_name = None, col_y5_name = None, col_y6_name = None, reference_line = None,  plot_boxplot = True, calculate_confidence_interval = True)

1) dataframe: dataframe no qual os dados se encontram

2) col_y1_name: nome da coluna (variável) na qual se encontram dados y que serão analisados com o ANOVA. Deve vir entre aspas e exatamente igual à forma como está escrito. 

        Exemplo: col_y1_name = "Y1"

3) col_y2_name: nome da coluna (variável) na qual se encontram dados y que serão analisados com o ANOVA. Deve vir entre aspas e exatamente igual à forma como está escrito. 

        Exemplo: col_y2_name = "Y2"

###### Para usar esta função, são necessários ao menos dois agrupamentoe de dados, os quais são inseridos em col_y1_name e col_y2_name, como explicado acima.

- Caso mais variáveis sejam analisadas, preencha, EM SEQUÊNCIA, col_y3_name, col_y4_name, col_y5_name e col_y6_name da mesma forma. Caso não use alguma das variáveis, mantenha ela em None.

        - se apenas 2 variáveis forem utilizadas, mantenha col_y3_name = None, col_y4_name = None, col_y5_name = None e col_y6_name = None

        - se 3 variáveis forem utilizadas, preencha col_y3_name e mantenha col_y4_name = None, col_y5_name = None e col_y6_name = None

        - se 4 variáveis forem utilizadas, preencha col_y3_name e col_y4_name, e mantenha col_y5_name = None e col_y6_name = None

        - se 5 variáveis forem utilizadas, preencha preencha col_y3_name, col_y4_name e col_y5_name, e mantenha col_y6_name = None

        - se 6 variáveis forem utilizadas, preencha todos os campos col_y3_name, col_y4_name, col_y5_name, e col_y6_name

4) confidence: nível de confiança para o cálculo do intervalo da média (ou seja, a probabilidade de a média se encontrar neste intervalo = confidence).
    
        Mantenha confidence = 0.95 para 95% de confiança
    
        - Caso deseje reduzir, basta alterar o valor: confidence = 0.90 fornece 90% de confiança
    
5) reference_line: este parâmetro colocará uma linha de referência horizontal no boxplot, permitindo a visualização se algum dos intervalos avaliados engloba ou não esta linha de referência (evidenciando se existe ou não elementos que permitam dizer que a média é ou não diferente daquele valor).

Declare como um valor numérico ou mantenha reference_line = None, caso não deseje incluir linhas. 

Caso haja linha de referência e cálculo do intervalo de confiança, o dataframe de saída informará se o valor de referência pertence ou não aos intervalos de confiança das médias. 

Exemplo:
    
        reference_line = 10.25 incluirá uma reta horizontal na altura Y = 10.25 do boxplot. 

Note que o número não deve vir entre aspas, e que o separador decimal é o ponto.

6) plot_boxplot: mantenha plot_boxplot = True, caso deseje que o boxplot seja gerado, ou mude para plot_boxplot = False, caso não deseje gerar o gráfico.

Note que apenas os valores True e False são aceitos, e eles não devem vir entre aspas.

7) calculate_confidence_interval: mantenha calculate_confidence_interval = True, caso deseje calcular o intervalo de confiança da média. 

###### Neste caso, além do p-valor do ANOVA (probabilidade de as médias serem iguais), o dataframe de saída mostrará os limites inferior e superior dos intervalos de confiança para o nível de confiança especificado em 'confidence'. 

Caso não deseje calcular os intervalos, mude para calculate_confidence_interval = False.

Note que apenas os valores True e False são aceitos, e eles não devem vir entre aspas.

###### t-Student
seja xbar a média de n medições, cujo desvio-padrão amostral é s

seja t(confidence; gl) a distribuição t-Student com confidence*100% de confiança e gl = n-1 graus de liberdade

t = (xbar +- mu)/(s/sqrt(n)), onde sqrt = square root = raiz quadrada

o intervalo de confiança da média será dado por:

xbar - ts/sqrt(n) <= mu <= xbar + ts/sqrt(n)

In [ ]:
## CÉLULA 12: ANÁLISE DE VARIÂNCIA, CÁLCULO DOS INTERVALOS DE CONFIANÇA E CONSTRUÇÃO DO BOXPLOT PARA SEUS DADOS

#DEFINA O NOME DA COLUNA COM A PRIMEIRA VARIÁVEL Y (Y1) comparada. 
#Basta substituir o valor entre aspas pelo nome da coluna onde está Y1:
nome_y1 = "SUBSTITUA ESTE TEXTO ENTRE ASPAS PELO NOME DA VARIÁVEL Y1 (Título da coluna)"

#DEFINA O NOME DA COLUNA COM A SEGUNDA VARIÁVEL Y (Y2) comparada. 
#Basta substituir o valor entre aspas pelo nome da coluna onde está Y2:
nome_y2 = "SUBSTITUA ESTE TEXTO ENTRE ASPAS PELO NOME DA VARIÁVEL Y2 (Título da coluna)"


confidence_interval = anova_boxplot_separate_columns(dataframe = dataset, col_y1_name = nome_y1, col_y2_name = nome_y2, confidence = 0.95, col_y3_name = None, col_y4_name = None, col_y5_name = None, col_y6_name = None, reference_line = None,  plot_boxplot = True, calculate_confidence_interval = True):

"""
Substitua os demais campos de acordo com as instruções dadas na CÉLULA 11.
- Os nomes das demais colunas devem ser declarados diretamente entre aspas. Por exemplo: col_y3_name = "NOME DA COLUNA Y3"
"""

confidence_interval

"""VOCÊ DESEJA EXPORTAR OS DADOS?
Caso deseje exportar os dados, copie a seguinte linha para o espaço não-vermelho após as aspas. Substitua o endereço pela
pasta onde você deseja salvar seu arquivo. Substitua int_confianca pelo nome que deseja para seu arquivo. Mantenha
a extensão csv

confidence_interval.to_csv(r"D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 3 - ANN classification\ANN.Class - 1 - user satisfaction\int_confianca.csv", index = False)

"""